In [ ]:
from benchmarklib import BenchmarkDatabase
from rbf import RandomBooleanFunctionTrial, RandomBooleanFunction
from benchmark import Trials
from benchmarklib.compilers import CompileType

In [ ]:
benchmark_db = BenchmarkDatabase("rbf-new.db", RandomBooleanFunction, RandomBooleanFunctionTrial)
old_db = Trials()

In [ ]:
with old_db._connect() as conn:
    count = conn.execute("SELECT COUNT(*) FROM trials").fetchone()[0]
    print(f"Number of trials in old db: {count}")

In [ ]:
with benchmark_db._connect() as conn:
    count = conn.execute("SELECT COUNT(*) FROM trials").fetchone()[0]
    print(f"Number of trials in new db: {count}")

In [ ]:
with benchmark_db._connect() as conn:
    latest = conn.execute("SELECT instance_id FROM problem_instances LIMIT 300").fetchall()
    print(f"Latest trial ID in new db: {latest}")

In [ ]:
done = set()

In [ ]:
def done_through(num_vars, complexity):
    for _num_vars in range(2, 33):
        for _complexity in range(1, 32):
            if _num_vars == num_vars and _complexity == complexity:
                done.add((_num_vars, _complexity))
                return
            done.add((_num_vars, _complexity))

done_through(14, 1)

In [ ]:
import time
for num_vars in range(2, 33):
    for complexity in range(1, 32):
        if (num_vars, complexity) in done:
            continue
        print(f"Processing num_vars={num_vars}, complexity={complexity}")
        data = old_db.get_per_statement(num_vars=num_vars, complexity=complexity)
        for statement, trials in data.items():
            print(f"  Statement: {statement} with {len(trials)} trials")
            existing_instances = benchmark_db.find_problem_instances(problem_data_filters={"statement": statement, "num_vars": num_vars, "complexity": complexity})
            if len(existing_instances) > 0:
                problem = existing_instances[0]
            else:
                problem = RandomBooleanFunction(statement=statement, num_vars=num_vars, complexity=complexity)
                problem.instance_id = benchmark_db.save_problem_instance(problem)

            for t in trials:
                trial = RandomBooleanFunctionTrial(
                    problem_instance=problem,
                    job_id=t.job_id,
                    job_pub_idx=t.job_pub_idx,
                    compiler_name=CompileType.CLASSICAL_FUNCTION,
                    simulation_counts=None,
                    counts=t.counts,
                    input_state=t.input_state,
                )
                benchmark_db.save_trial(trial)
        done.add((num_vars, complexity))
        print(f"Done processing num_vars={num_vars}, complexity={complexity}")
        #time.sleep(2)